# sub_zy_single_20

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# load data

In [2]:
path='./data/'   #复赛数据地址
train_sale_path=path+'train_sales_data.csv'
train_sale=pd.read_csv(train_sale_path)
train_search_path=path+'train_search_data.csv'
train_search=pd.read_csv(train_search_path)
train_reply_path=path+'train_user_reply_data.csv'
train_reply=pd.read_csv(train_reply_path)
test_path=path+'evaluation_public.csv'
test=pd.read_csv(test_path)

In [3]:
path = '../chusai/data/'   #初赛数据地址
round1_data_path=path+'train_sales_data.csv'
round1_data=pd.read_csv(round1_data_path)
round1_model=list(round1_data['model'].unique())

In [4]:
#合并训练集
train_data=pd.merge(train_sale,train_search,how='left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
train_data=train_data.merge(train_reply,how='left',on=['model','regYear','regMonth'])

#打标
train_data['label']=train_data['salesVolume']
del train_data['salesVolume'],test['forecastVolum']

#编码
transfer=train_data.drop_duplicates('model').set_index('model')['bodyType']
train_data['bodyType'] = train_data['model'].map(transfer)
test['bodyType']=test['model'].map(transfer)
for i in ['bodyType', 'model']:
    train_data[i+'or']=train_data[i]
    test[i+'or']=test[i]
    t=dict(zip(train_data[i].unique(), range(train_data[i].nunique())))
    train_data[i] = train_data[i].map(t)
    test[i]= test[i].map(t)
train=train_data

# 编码

In [5]:
train['mt'] = (train['regYear'] - 2016) * 12 + train['regMonth']
test['mt']= (test['regYear'] - 2016) * 12 + test['regMonth']

'编码便于提取特征'
train['model_adcode'] = train['adcode'] + train['model']   
test['model_adcode'] = test['adcode'] + test['model']

train['adcode_bodyType'] = train['adcode']* 1000 + train['bodyType']   
test['adcode_bodyType'] = test['adcode']* 1000 + test['bodyType']

train['model_adcode_regMonth'] = train['model_adcode'] * 100 + train['regMonth']      #省份+车型+月份 编码
test['model_adcode_regMonth']= test['model_adcode'] * 100 + test['regMonth']      #省份+车型+月份 编码

train['model_regMonth'] = train['model'] * 1000 + train['regMonth']      #车型+月份 编码
test['model_regMonth']= test['model'] * 1000 + test['regMonth']      #车型+月份 编码

train['adcode_regMonth'] = train['adcode'] * 1000 + train['regMonth']      #省份+月份 编码
test['adcode_regMonth']= test['adcode'] * 1000 + test['regMonth']      #省份+月份 编码

train['bodyType_regMonth'] = train['bodyType'] * 1000 + train['regMonth']      #车身+月份 编码
test['bodyType_regMonth']= test['bodyType'] * 1000 + test['regMonth']      #车身+月份 编码


train['adcode_regMonth_bodyType'] = train['adcode_bodyType'] * 1000 + train['regMonth']      #省份+月份 编码
test['adcode_regMonth_bodyType']= test['adcode_bodyType'] * 1000 + test['regMonth']      #省份+月份 编码

train['model_adcode_mt'] = train['model_adcode'] * 100 + train['mt']      #省份+车型+月份 编码
test['model_adcode_mt']= test['model_adcode'] * 100 + test['mt']      #省份+车型+月份 编码

#季度
def get_quar(x):
    if x<=3:
        return 1
    elif x>3 and x<=6:
        return 2 
    elif x>6 and x<=9:
        return 3
    elif x>9:
        return 12
train['quarter']=train['regMonth'].apply(lambda x:get_quar(x))
test['quarter']=test['regMonth'].apply(lambda x:get_quar(x))
train['model_quarter'] = train['quarter']*10000 + train['model']   
test['model_quarter'] = test['quarter'] *10000+ test['model']


#前n月是哪一月
def get_last_n_month(x,n):
    if x<=n:
        return 12+x-n
    else:
        return x-n

for i in range(1,7):
    train['last_month_{}'.format(i)]=train['regMonth'].apply(lambda x:get_last_n_month(x,i))
    test['last_month_{}'.format(i)]=test['regMonth'].apply(lambda x:get_last_n_month(x,i))

#后n月是哪一月
def get_next_n_month(x,n):
    if x>12-n:
        return x-12+n
    else:
        return x+n

for i in  range(1,7):
    train['next_month_{}'.format(i)]=train['regMonth'].apply(lambda x:get_next_n_month(x,i))
    test['next_month_{}'.format(i)]=test['regMonth'].apply(lambda x:get_next_n_month(x,i))


#销售淡季与旺季
def get_nice_month(x):
    if x >=5 and x<=8:
        return 0
    if x>=10 or x==1:
        return 1
    return 0

train['nice_month']=train['regMonth'].apply(lambda x:get_nice_month(x))
test['nice_month']=test['regMonth'].apply(lambda x:get_nice_month(x))

def get_spring_counter(year,month):
    if year==2016:
        if month==2:
            return 0
        if month==1:
            return -1
        return month-2
    if year==2017:
        if month==1:
            return 0
        return month-1  
    if year==2018:
        if month==2:
            return 0
        if month==1:
            return -1
        return month-2  

train['spring_counter']=list(map(lambda x,y:get_spring_counter(x,y),train['regYear'],train['regMonth']))
test['spring_counter']=list(map(lambda x,y:get_spring_counter(x,y),test['regYear'],test['regMonth']))   


def is_conf_year(x):
    if x>=1 and x<=6:
        return 1
    return 0

train['con_month']=train['regMonth'].apply(lambda x:is_conf_year(x))
test['con_month']=test['regMonth'].apply(lambda x:is_conf_year(x))

test['label']=np.nan   #预测结果
train=pd.concat([train,test],axis=0)   

cate_feat = ['adcode', 'bodyType', 'model', 'regMonth','spring_counter']+['next_month_{}'.format(i) for i in range(1,7)]+[
    'last_month_{}'.format(i) for i in  range(1,7)]


train_help=train.copy()

'编码便于提取特征'

In [6]:
train=train[~train['modelor'].isin(round1_model)]
test=test[~test['modelor'].isin(round1_model)]
train_help=train_help[~train_help['modelor'].isin(round1_model)]

In [7]:
train.shape

(13552, 39)

In [8]:
test.shape

(1936, 36)

In [9]:
train_help.shape

(13552, 39)

# 模型函数

In [10]:
import lightgbm as lgb
# import xgboost as xgb

def get_model_type(train_x,train_y,valid_x,valid_y,num_feat,cate_feat,m_type='lgb',num=1000):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='rmse',
                                max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2020,
                                n_estimators=num, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              categorical_feature=cate_feat, 
              early_stopping_rounds=100, verbose=100)
        
        
        f_importance = pd.DataFrame(columns=['f_name', 'importance'])
        f_importance['importance'] = model.feature_importances_
        f_importance['f_name'] =num_feat+cate_feat
        f_importance['f_score'] = f_importance['importance'] / f_importance['importance'].sum()
        f_importance.sort_values(['f_score'], ascending=False, inplace=True)
        f_importance.drop(columns=['importance'], axis=1, inplace=True)
#         f_importance.to_csv('im_lgb.csv',index=False)
        print(f_importance)
        
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.05, n_estimators=num, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)    
    return model

from sklearn.metrics import mean_squared_error as mse
'求线下结果'
def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred: list,
        label: [list, 'mean'],

        }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    return 1 - np.mean(nrmse_score)

'label 变换函数'
def confer_label(y):
    return np.log(y+1)

def confer_back_label(y):
    return  np.power(np.exp(1),y)-1

'求线下结果'

'label 变换函数'

# 1月模型

## 划分区间

In [11]:
'1月   验证集是9月'
num_feat=[]

train_set=[]       #训练区间
train_his_set=[]   #历史区间
train_y_set=[]     #训练区间label


gap=8
shift=0
for i in range(gap+shift+1,21):
    train_his_set.append(train[(train['mt'].between(i-gap,i-1))])
    train_set.append(train[(train['mt']==i)])
    train_y_set.append(train[(train['mt']==i)][['label']])

val_set=[]       #训练区间
val_his_set=[]   #历史区间
val_y_set=[]     #训练区间label

for i in [21]:
    val_his_set.append(train[(train['mt'].between(i-gap,i-1))])
    val_set.append(train[(train['mt']==i)])
    val_y_set.append(train[(train['mt']==i)][['label']])    

test_his_set=[]    #test历史区间
test_set=[]        #test训练区间
for i in [25]:
    test_his_set.append(train[(train['mt'].between(i-gap,i-1))])
    test_set.append(train[(train['mt']==i)])
    

#train_y
train_y=pd.concat([i for i in train_y_set],axis=0,sort=True)['label'].values
train_y=confer_label(train_y)

#val_y
val_y=pd.concat([i for i in val_y_set],axis=0,sort=True)['label'].values
val_y=confer_label(val_y)


#val_preds
val_pred_1=val_set[0][['model','label']]
test_preds=test[['id','mt']]

'1月   验证集是9月'

## 1月特征

In [12]:
def get_his_features_1(his_or,data_or):
    data=data_or.copy()
    his=his_or.copy()
    
    prefiex='his_'
    
    his['label']=confer_label(his['label'].values)
    
    mt=list(his.mt.unique())
    print(mt)
    

    sum_columns=['model_adcode']
    
    
    def get_last_key_sum(data_,i,key):
        data=data_.copy()
        temp=his[his['mt']==mt[i]]
        for col in sum_columns:
            f=temp[[col,key]].groupby(col)[key].agg('sum')
            data[prefiex+'{}_all_last{}_'.format(key,-i)+col]=data[col].map(f)
        return data
    
    
    
    '过去月的销量'   
    for i in range(1,len(mt)+1):
        data= get_last_key_sum(data,-i,'label') 
        
    
        
    '过去月的涨幅'    
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,1)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
    
    '过去月的涨幅是正还是负'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去月的涨幅有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all'.format(key)+col]=all_tag_1
        
    '过去月的差分'
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
            
    '过去月差分的tag'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
                
    '过去一阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff'.format(key)+col]=all_tag_1 
                
    '过去月的二阶差分'
    for col in sum_columns:
        for i in range(1,len(mt)-1):
            data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_increased_diff'.format(key,i+1)+col].values  
    '过去月二阶差分的tag'    
    for col in sum_columns:
        for i in range(1,len(mt)-1):
              data[prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去二阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)-1):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff2'.format(key)+col]=all_tag_1 
                    
    


    '销售最大最小'
    key='label'
    for col in sum_columns:
        pre=prefiex+'sales_'+col
        temp=his[[col,key]]
        tmpdf = temp.groupby(col, as_index=False, sort=False)[key].agg(
                        {pre+ '_mean': 'mean',
                         pre + '_max': 'max', pre + '_min': 'min'})
        data=data.merge(tmpdf,on=col,how='left')
        
    
    '过去月的popu'
    def get_last_key_sum(data_,i,key):
        data=data_.copy()
        temp=his[his['mt']==mt[i]]
        for col in sum_columns:
            f=temp[[col,key]].groupby(col)[key].agg('sum')
            data[prefiex+'{}_all_last{}_'.format(key,-i)+col]=data[col].map(f)
        return data
    
    for i in range(1,len(mt)+1):
            data= get_last_key_sum(data,-i,'popularity')
        

    key='popularity'
    def get_last_key_sum(data_,i,key):
        data=data_.copy()
        temp=his[his['mt']==mt[i]]
        for col in sum_columns:
            f=temp[[col,key]].groupby(col)[key].agg('sum')
            data[prefiex+'{}_all_last{}_'.format(key,-i)+col]=data[col].map(f)
        return data
    
    for i in range(1,len(mt)+1):
            data= get_last_key_sum(data,-i,'popularity')
    
    '过去月的涨幅popu'
    key='popularity'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,1)+col].values-data[
                prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
            
    '过去月的涨幅是正还是负'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_tag_'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去月的差分popu'
    key='popularity'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_gap'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,i)+col].values-data[
                prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
    
    
    '上n月是增加还是减少'
    key='popularity'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_gap_increased_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_gap'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
        
        
    '过去月的涨幅有多少为正的popularity'
    key='popularity'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_gap_increased_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_last{}_gap_incread_tag1_all'.format(key,i)+col]=all_tag_1
    
    
    return data

In [13]:
def get_all_features(test_his_set,test_set,train_his_set,train_set,val_his_set,val_set,get_his_features,num_feat_list,cate_feat):
     
    num_feat=num_feat_list.copy()
    
    test1_features1=get_his_features(test_his_set[0],test_set[0])
    for col in list(test1_features1.columns):
        if re.match('his', col)!=None:
            num_feat.append(col)
    
    #test
    test_features_set=[]
    for i in tqdm(range(0,len(test_set))):
        test_features_set.append(get_his_features(test_his_set[i],test_set[i])[num_feat])


    #train
    train_features_set=[]
    for i in tqdm(range(0,len(train_set))):
        train_features_set.append(get_his_features(train_his_set[i],train_set[i])[num_feat])

    #val
    val_features_set=[]
    for i in tqdm(range(0,len(val_set))):
        val_features_set.append(get_his_features(val_his_set[i],val_set[i])[num_feat])
    
    'cat features'
    #test
    test_features_set2=[]
    for i in tqdm(range(0,len(test_set))):
        test_features2=test_set[i][cate_feat]
        test_features_set2.append(test_features2)

    #train
    train_features_set2=[]
    for i in tqdm(range(0,len(train_set))):
        train_features2=train_set[i][cate_feat]
        train_features_set2.append(train_features2)    
    
    #val
    val_features_set2=[]
    for i in tqdm(range(0,len(val_set))):
        val_features2=val_set[i][cate_feat]
        val_features_set2.append(val_features2)    

    'concat features'
    #test
    
    test_features1=pd.concat([i for i in test_features_set],axis=0,sort=True)
    test_features2=pd.concat([i for i in test_features_set2],axis=0,sort=True)
    test_features1.index=np.arange(len(test_features1))
    test_features2.index=np.arange(len(test_features2))
    test1_x=pd.concat([test_features1,test_features2],axis=1)
    print(test1_x.shape)
    #train
    train_features1=pd.concat([i for i in train_features_set],axis=0,sort=True)
    train_features2=pd.concat([i for i in train_features_set2],axis=0,sort=True)
    train_features1.index=np.arange(len(train_features1))
    train_features2.index=np.arange(len(train_features2))
    train_x=pd.concat([train_features1,train_features2],axis=1)[list(test1_x.columns)]
    print(train_x.shape)
    #val
    val_features1 = pd.concat([i for i in val_features_set], axis=0, sort=True)
    val_features2 = pd.concat([i for i in val_features_set2], axis=0, sort=True)
    val_features1.index = np.arange(len(val_features1))
    val_features2.index = np.arange(len(val_features2))
    val_x = pd.concat([val_features1, val_features2], axis=1)[list(test1_x.columns)]
    print(val_x.shape)
    
    
    for i in cate_feat:
        train_x[i]=train_x[i].astype('category')
        test1_x[i]=test1_x[i].astype('category')
        val_x[i]=val_x[i].astype('category')
    
    return train_x,val_x,test1_x,num_feat


num_feat=[]
cate_feat=cate_feat
# cate_feat =cate_feat+ ['last_month_{}'.format(i) for i in range(1,7)]
train_x,val_x,test_x,num_feat_this=get_all_features(test_his_set,test_set,train_his_set,
                                              train_set,val_his_set,val_set,get_his_features_1,num_feat,cate_feat)

  0%|                                                    | 0/1 [00:00<?, ?it/s]

[17, 18, 19, 20, 22, 23, 21, 24]
[17, 18, 19, 20, 22, 23, 21, 24]


  8%|███▌                                       | 1/12 [00:00<00:01,  6.40it/s]

[1, 2, 3, 4, 5, 6, 7, 8]
[2, 3, 4, 5, 6, 7, 8, 9]


 25%|██████████▊                                | 3/12 [00:00<00:01,  6.74it/s]

...

[10, 11, 12, 13, 14, 15, 16, 17]
[11, 12, 13, 14, 15, 16, 17, 18]


  0%|                                                    | 0/1 [00:00<?, ?it/s]

[12, 13, 14, 15, 16, 17, 18, 19]
[13, 14, 15, 16, 17, 18, 19, 20]


100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


(484, 108)
(5808, 108)
(484, 108)


## 1月模型

In [14]:
lgb_model=get_model_type(train_x,train_y,val_x,val_y,num_feat_this,cate_feat)
#线下分数
val_pred_1['pred_label']= confer_back_label(lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration_))
# np.power(np.exp(1),lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration_))-1
train_help.loc[(train_help['mt']==21),'label']=val_pred_1['pred_label'].values    #方便验证
print('9月验证：',score(val_pred_1))
#预测一月
test.loc[(test['mt']==25),'label'] =  confer_back_label(lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration_))
# np.power(np.exp(1),lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration_))-1
train.loc[(train['mt']==25),'label']=test.loc[(test['mt']==25),'label'].values
print('1月均值',test.loc[(test['mt']==25),'label'].mean())

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.187005	valid_1's rmse: 0.210199
[200]	training's rmse: 0.137451	valid_1's rmse: 0.211967
Early stopping, best iteration is:
[116]	training's rmse: 0.175139	valid_1's rmse: 0.209826
                                               f_name   f_score
93                                              model  0.117241
9           his_label_all_last2_increasedmodel_adcode  0.095402
99                                       next_month_4  0.085632
91                                             adcode  0.044540
6                    his_label_all_last7_model_adcode  0.032471
..                                                ...       ...
33  his_label_all_last4_increased_diff_tagmodel_ad...  0.000000
68     his_popularity_all_last7_increasedmodel_adcode  0.000000
35  his_label_all_last6_increased_diff_tagmodel_ad...  0.000000
73  his_popularity_all_last5_increased_tag_model_a...  0.000000
82           his_popularity

# 2月模型

## 划分区间

In [15]:
'2月   验证集是10月'
num_feat=[]
# cate_feat = ['adcode', 'bodyType', 'model', 'regMonth','spring_counter']

train_set=[]       #训练区间
train_his_set=[]   #历史区间
train_y_set=[]     #训练区间label


gap=8
shift=0
for i in range(gap+shift+1,21):
    train_his_set.append(train[(train['mt'].between(i-gap-shift,i-shift-1))])
    train_set.append(train[(train['mt']==i)])
    train_y_set.append(train[(train['mt']==i)][['label']])

val_set=[]       #训练区间
val_his_set=[]   #历史区间
val_y_set=[]     #训练区间label

for i in [22]:
    val_his_set.append(train_help[(train_help['mt'].between(i-gap-shift,i-shift-1))])
    val_set.append(train_help[(train_help['mt']==i)])
    val_y_set.append(train_help[(train_help['mt']==i)][['label']])    

test_his_set=[]    #test历史区间
test_set=[]        #test训练区间
for i in [26]:
    test_his_set.append(train[(train['mt'].between(i-gap-shift,i-shift-1))])
    test_set.append(train[(train['mt']==i)])
    

#train_y
train_y=pd.concat([i for i in train_y_set],axis=0,sort=True)['label'].values
train_y=confer_label(train_y)

#val_y
val_y=pd.concat([i for i in val_y_set],axis=0,sort=True)['label'].values
val_y=confer_label(val_y)


#val_preds
val_pred_2=val_set[0][['model','label']]

'2月   验证集是10月'

## 2月特征

In [16]:
def get_his_features_1(his_or,data_or):
    data=data_or.copy()
    his=his_or.copy()
    
    prefiex='his_'
    
    his['label']=confer_label(his['label'].values)
    
    mt=list(his.mt.unique())
    print(mt)
    

    sum_columns=['model_adcode']
    
    
    def get_last_key_sum(data_,i,key):
        data=data_.copy()
        temp=his[his['mt']==mt[i]]
        for col in sum_columns:
            f=temp[[col,key]].groupby(col)[key].agg('sum')
            data[prefiex+'{}_all_last{}_'.format(key,-i)+col]=data[col].map(f)
        return data
    
    '过去月的销量'   
    for i in range(1,len(mt)+1):
        data= get_last_key_sum(data,-i,'label') 
        
    '过去月的涨幅'    
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,1)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
    
    '过去月的涨幅是正还是负'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去月的涨幅有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all'.format(key)+col]=all_tag_1
        
    '过去月的差分'
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
            
    '过去月差分的tag'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
                
    '过去一阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff'.format(key)+col]=all_tag_1 
                
    '过去月的二阶差分'
    for col in sum_columns:
        for i in range(1,len(mt)-1):
            data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_increased_diff'.format(key,i+1)+col].values  
    '过去月二阶差分的tag'    
    for col in sum_columns:
        for i in range(1,len(mt)-1):
              data[prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去二阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)-1):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff2'.format(key)+col]=all_tag_1 
                    
    


    '销售最大最小'
    key='label'
    for col in sum_columns:
        pre=prefiex+'sales_'+col
        temp=his[[col,key]]
        tmpdf = temp.groupby(col, as_index=False, sort=False)[key].agg(
                        {pre+ '_mean': 'mean',
                         pre + '_max': 'max', pre + '_min': 'min'})
        data=data.merge(tmpdf,on=col,how='left')
        
    
    return data

In [17]:
def get_all_features(test_his_set,test_set,train_his_set,train_set,val_his_set,val_set,get_his_features,num_feat_list,cate_feat):
     
    num_feat=num_feat_list.copy()
    
    test1_features1=get_his_features(test_his_set[0],test_set[0])
    for col in list(test1_features1.columns):
        if re.match('his', col)!=None:
            num_feat.append(col)
    
    #test
    test_features_set=[]
    for i in tqdm(range(0,len(test_set))):
        test_features_set.append(get_his_features(test_his_set[i],test_set[i])[num_feat])


    #train
    train_features_set=[]
    for i in tqdm(range(0,len(train_set))):
        train_features_set.append(get_his_features(train_his_set[i],train_set[i])[num_feat])

    #val
    val_features_set=[]
    for i in tqdm(range(0,len(val_set))):
        val_features_set.append(get_his_features(val_his_set[i],val_set[i])[num_feat])
    
    'cat features'
    #test
    test_features_set2=[]
    for i in tqdm(range(0,len(test_set))):
        test_features2=test_set[i][cate_feat]
        test_features_set2.append(test_features2)

    #train
    train_features_set2=[]
    for i in tqdm(range(0,len(train_set))):
        train_features2=train_set[i][cate_feat]
        train_features_set2.append(train_features2)    
    
    #val
    val_features_set2=[]
    for i in tqdm(range(0,len(val_set))):
        val_features2=val_set[i][cate_feat]
        val_features_set2.append(val_features2)    

    'concat features'
    #test
    
    test_features1=pd.concat([i for i in test_features_set],axis=0,sort=True)
    test_features2=pd.concat([i for i in test_features_set2],axis=0,sort=True)
    test_features1.index=np.arange(len(test_features1))
    test_features2.index=np.arange(len(test_features2))
    test1_x=pd.concat([test_features1,test_features2],axis=1)
    print(test1_x.shape)
    #train
    train_features1=pd.concat([i for i in train_features_set],axis=0,sort=True)
    train_features2=pd.concat([i for i in train_features_set2],axis=0,sort=True)
    train_features1.index=np.arange(len(train_features1))
    train_features2.index=np.arange(len(train_features2))
    train_x=pd.concat([train_features1,train_features2],axis=1)[list(test1_x.columns)]
    print(train_x.shape)
    #val
    val_features1 = pd.concat([i for i in val_features_set], axis=0, sort=True)
    val_features2 = pd.concat([i for i in val_features_set2], axis=0, sort=True)
    val_features1.index = np.arange(len(val_features1))
    val_features2.index = np.arange(len(val_features2))
    val_x = pd.concat([val_features1, val_features2], axis=1)[list(test1_x.columns)]
    print(val_x.shape)
    
    
    for i in cate_feat:
        train_x[i]=train_x[i].astype('category')
        test1_x[i]=test1_x[i].astype('category')
        val_x[i]=val_x[i].astype('category')
    
    return train_x,val_x,test1_x,num_feat


num_feat=[]
cate_feat=cate_feat
# cate_feat =cate_feat+ ['last_month_{}'.format(i) for i in range(1,7)]
train_x,val_x,test_x,num_feat_this=get_all_features(test_his_set,test_set,train_his_set,
                                              train_set,val_his_set,val_set,get_his_features_1,num_feat,cate_feat)

  0%|                                                   | 0/12 [00:00<?, ?it/s]

[18, 19, 20, 22, 23, 21, 24, 25]
[18, 19, 20, 22, 23, 21, 24, 25]
[1, 2, 3, 4, 5, 6, 7, 8]


 17%|███████▏                                   | 2/12 [00:00<00:00, 11.64it/s]

[2, 3, 4, 5, 6, 7, 8, 9]
[3, 4, 5, 6, 7, 8, 9, 10]
[4, 5, 6, 7, 8, 9, 10, 11]


 50%|█████████████████████▌                     | 6/12 [00:00<00:00, 12.45it/s]

...

[8, 9, 10, 11, 12, 13, 14, 15]
[9, 10, 11, 12, 13, 14, 15, 16]
[10, 11, 12, 13, 14, 15, 16, 17]


  0%|                                                    | 0/1 [00:00<?, ?it/s]

[11, 12, 13, 14, 15, 16, 17, 18]
[12, 13, 14, 15, 16, 17, 18, 19]
[14, 15, 16, 17, 18, 19, 20, 21]


100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


(484, 71)
(5808, 71)
(484, 71)


## 2月模型

In [18]:
lgb_model=get_model_type(train_x,train_y,val_x,val_y,num_feat_this,cate_feat)
#线下分数
val_pred_2['pred_label']=confer_back_label(lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration_))
train_help.loc[(train_help['mt']==22),'label']=val_pred_2['pred_label'].values  #方便验证
print('10月验证：',score(val_pred_2))
#预测一月
test.loc[(test['mt']==26),'label'] =  confer_back_label(lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration_))
train.loc[(train['mt']==26),'label']=test.loc[(test['mt']==26),'label'].values
print('2月均值',test.loc[(test['mt']==26),'label'].mean())

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.193324	valid_1's rmse: 0.285221
[200]	training's rmse: 0.143139	valid_1's rmse: 0.28506
Early stopping, best iteration is:
[141]	training's rmse: 0.166127	valid_1's rmse: 0.283635
                                               f_name   f_score
56                                              model  0.112057
9           his_label_all_last2_increasedmodel_adcode  0.099054
62                                       next_month_4  0.095272
54                                             adcode  0.058392
6                    his_label_all_last7_model_adcode  0.041135
..                                                ...       ...
39    his_label_all_last2_increased_diff2model_adcode  0.000000
14          his_label_all_last7_increasedmodel_adcode  0.000000
18      his_label_all_last4_increased_tagmodel_adcode  0.000000
47  his_label_all_last4_increased_diff_tag2model_a...  0.000000
35  his_label_all_last6_incr

# 3月模型

## 划分区间

In [19]:
'3月   验证集是11月'
num_feat=[]
# cate_feat = ['adcode', 'bodyType', 'model', 'regMonth','spring_counter']

train_set=[]       #训练区间
train_his_set=[]   #历史区间
train_y_set=[]     #训练区间label


gap=8
shift=0
for i in range(gap+shift+1,21):
    train_his_set.append(train[(train['mt'].between(i-gap-shift,i-shift-1))])
    train_set.append(train[(train['mt']==i)])
    train_y_set.append(train[(train['mt']==i)][['label']])

val_set=[]       #训练区间
val_his_set=[]   #历史区间
val_y_set=[]     #训练区间label

for i in [23]:
    val_his_set.append(train_help[(train_help['mt'].between(i-gap-shift,i-shift-1))])
    val_set.append(train_help[(train_help['mt']==i)])
    val_y_set.append(train_help[(train_help['mt']==i)][['label']])    

test_his_set=[]    #test历史区间
test_set=[]        #test训练区间
for i in [27]:
    test_his_set.append(train[(train['mt'].between(i-gap-shift,i-shift-1))])
    test_set.append(train[(train['mt']==i)])
    

#train_y
train_y=pd.concat([i for i in train_y_set],axis=0,sort=True)['label'].values
train_y=confer_label(train_y)

#val_y
val_y=pd.concat([i for i in val_y_set],axis=0,sort=True)['label'].values
val_y=confer_label(val_y)


#val_preds
val_pred_3=val_set[0][['model','label']]

'3月   验证集是11月'

## 3月特征

In [20]:
def get_his_features_1(his_or,data_or):
    data=data_or.copy()
    his=his_or.copy()
    
    prefiex='his_'
    
    his['label']=confer_label(his['label'].values)
    
    mt=list(his.mt.unique())
    print(mt)
    

    sum_columns=['model_adcode']
    
    
    def get_last_key_sum(data_,i,key):
        data=data_.copy()
        temp=his[his['mt']==mt[i]]
        for col in sum_columns:
            f=temp[[col,key]].groupby(col)[key].agg('sum')
            data[prefiex+'{}_all_last{}_'.format(key,-i)+col]=data[col].map(f)
        return data
    
    '过去月的销量'   
    for i in range(1,len(mt)+1):
        data= get_last_key_sum(data,-i,'label') 
        
    '过去月的涨幅'    
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,1)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
    
    '过去月的涨幅是正还是负'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去月的涨幅有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all'.format(key)+col]=all_tag_1
        
    '过去月的差分'
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
            
    '过去月差分的tag'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
                
    '过去一阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff'.format(key)+col]=all_tag_1 
                
    '过去月的二阶差分'
    for col in sum_columns:
        for i in range(1,len(mt)-1):
            data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_increased_diff'.format(key,i+1)+col].values  
    '过去月二阶差分的tag'    
    for col in sum_columns:
        for i in range(1,len(mt)-1):
              data[prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去二阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)-1):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff2'.format(key)+col]=all_tag_1 
                    
    


    '销售最大最小'
    key='label'
    for col in sum_columns:
        pre=prefiex+'sales_'+col
        temp=his[[col,key]]
        tmpdf = temp.groupby(col, as_index=False, sort=False)[key].agg(
                        {pre+ '_mean': 'mean',
                         pre + '_max': 'max', pre + '_min': 'min'})
        data=data.merge(tmpdf,on=col,how='left')  
    
    return data

In [21]:
def get_all_features(test_his_set,test_set,train_his_set,train_set,val_his_set,val_set,get_his_features,num_feat_list,cate_feat):
     
    num_feat=num_feat_list.copy()
    
    test1_features1=get_his_features(test_his_set[0],test_set[0])
    for col in list(test1_features1.columns):
        if re.match('his', col)!=None:
            num_feat.append(col)
    
    #test
    test_features_set=[]
    for i in tqdm(range(0,len(test_set))):
        test_features_set.append(get_his_features(test_his_set[i],test_set[i])[num_feat])


    #train
    train_features_set=[]
    for i in tqdm(range(0,len(train_set))):
        train_features_set.append(get_his_features(train_his_set[i],train_set[i])[num_feat])

    #val
    val_features_set=[]
    for i in tqdm(range(0,len(val_set))):
        val_features_set.append(get_his_features(val_his_set[i],val_set[i])[num_feat])
    
    'cat features'
    #test
    test_features_set2=[]
    for i in tqdm(range(0,len(test_set))):
        test_features2=test_set[i][cate_feat]
        test_features_set2.append(test_features2)

    #train
    train_features_set2=[]
    for i in tqdm(range(0,len(train_set))):
        train_features2=train_set[i][cate_feat]
        train_features_set2.append(train_features2)    
    
    #val
    val_features_set2=[]
    for i in tqdm(range(0,len(val_set))):
        val_features2=val_set[i][cate_feat]
        val_features_set2.append(val_features2)    

    'concat features'
    #test
    
    test_features1=pd.concat([i for i in test_features_set],axis=0,sort=True)
    test_features2=pd.concat([i for i in test_features_set2],axis=0,sort=True)
    test_features1.index=np.arange(len(test_features1))
    test_features2.index=np.arange(len(test_features2))
    test1_x=pd.concat([test_features1,test_features2],axis=1)
    print(test1_x.shape)
    #train
    train_features1=pd.concat([i for i in train_features_set],axis=0,sort=True)
    train_features2=pd.concat([i for i in train_features_set2],axis=0,sort=True)
    train_features1.index=np.arange(len(train_features1))
    train_features2.index=np.arange(len(train_features2))
    train_x=pd.concat([train_features1,train_features2],axis=1)[list(test1_x.columns)]
    print(train_x.shape)
    #val
    val_features1 = pd.concat([i for i in val_features_set], axis=0, sort=True)
    val_features2 = pd.concat([i for i in val_features_set2], axis=0, sort=True)
    val_features1.index = np.arange(len(val_features1))
    val_features2.index = np.arange(len(val_features2))
    val_x = pd.concat([val_features1, val_features2], axis=1)[list(test1_x.columns)]
    print(val_x.shape)
    
    
    for i in cate_feat:
        train_x[i]=train_x[i].astype('category')
        test1_x[i]=test1_x[i].astype('category')
        val_x[i]=val_x[i].astype('category')
    
    return train_x,val_x,test1_x,num_feat


num_feat=[]
cate_feat=cate_feat
# cate_feat =cate_feat+ ['last_month_{}'.format(i) for i in range(1,7)]
train_x,val_x,test_x,num_feat_this=get_all_features(test_his_set,test_set,train_his_set,
                                              train_set,val_his_set,val_set,get_his_features_1,num_feat,cate_feat)

  0%|                                                   | 0/12 [00:00<?, ?it/s]

[19, 20, 22, 23, 21, 24, 25, 26]
[19, 20, 22, 23, 21, 24, 25, 26]
[1, 2, 3, 4, 5, 6, 7, 8]


 17%|███████▏                                   | 2/12 [00:00<00:00, 11.64it/s]

[2, 3, 4, 5, 6, 7, 8, 9]
[3, 4, 5, 6, 7, 8, 9, 10]
[4, 5, 6, 7, 8, 9, 10, 11]


 50%|█████████████████████▌                     | 6/12 [00:00<00:00, 12.20it/s]

...

[8, 9, 10, 11, 12, 13, 14, 15]
[9, 10, 11, 12, 13, 14, 15, 16]
[10, 11, 12, 13, 14, 15, 16, 17]


  0%|                                                    | 0/1 [00:00<?, ?it/s]

[11, 12, 13, 14, 15, 16, 17, 18]
[12, 13, 14, 15, 16, 17, 18, 19]
[15, 16, 17, 18, 19, 20, 22, 21]


100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


(484, 71)
(5808, 71)
(484, 71)


## 3月模型

In [22]:
lgb_model=get_model_type(train_x,train_y,val_x,val_y,num_feat_this,cate_feat)
#线下分数
val_pred_3['pred_label']=confer_back_label(lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration_))
train_help.loc[(train_help['mt']==23),'label']=val_pred_3['pred_label'].values  #方便验证
print('11月验证：',score(val_pred_3))
#预测一月
test.loc[(test['mt']==27),'label'] = confer_back_label(lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration_))
train.loc[(train['mt']==27),'label']=test.loc[(test['mt']==27),'label'].values
print('3月均值',test.loc[(test['mt']==27),'label'].mean())

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.193324	valid_1's rmse: 0.396262
[200]	training's rmse: 0.143139	valid_1's rmse: 0.391723
[300]	training's rmse: 0.117744	valid_1's rmse: 0.390038
[400]	training's rmse: 0.0987606	valid_1's rmse: 0.388725
Early stopping, best iteration is:
[392]	training's rmse: 0.100056	valid_1's rmse: 0.388281
                                             f_name   f_score
62                                     next_month_4  0.073129
56                                            model  0.064116
54                                           adcode  0.059269
9         his_label_all_last2_increasedmodel_adcode  0.048129
6                  his_label_all_last7_model_adcode  0.036905
..                                              ...       ...
64                                     next_month_6  0.000000
63                                     next_month_5  0.000000
39  his_label_all_last2_increased_diff2model_adcode  0.000

# 4月模型

## 划分区间

In [23]:
'1月'
num_feat=[]
# cate_feat = ['adcode', 'bodyType', 'model', 'regMonth','spring_counter']

train_set=[]       #训练区间
train_his_set=[]   #历史区间
train_y_set=[]     #训练区间label


gap=8
shift=0
for i in range(gap+shift+1,21):
    train_his_set.append(train[(train['mt'].between(i-gap-shift,i-shift-1))])
    train_set.append(train[(train['mt']==i)])
    train_y_set.append(train[(train['mt']==i)][['label']])

val_set=[]       #训练区间
val_his_set=[]   #历史区间
val_y_set=[]     #训练区间label

for i in [24]:
    val_his_set.append(train_help[(train_help['mt'].between(i-gap-shift,i-shift-1))])
    val_set.append(train_help[(train_help['mt']==i)])
    val_y_set.append(train_help[(train_help['mt']==i)][['label']])    

test_his_set=[]    #test历史区间
test_set=[]        #test训练区间
for i in [28]:
    test_his_set.append(train[(train['mt'].between(i-gap-shift,i-shift-1))])
    test_set.append(train[(train['mt']==i)])
    

#train_y
train_y=pd.concat([i for i in train_y_set],axis=0,sort=True)['label'].values
train_y=confer_label(train_y)

#val_y
val_y=pd.concat([i for i in val_y_set],axis=0,sort=True)['label'].values
val_y=confer_label(val_y)


#val_preds
val_pred_4=val_set[0][['model','label']]

'1月'

## 4月特征

In [24]:
def get_his_features_1(his_or,data_or):
    data=data_or.copy()
    his=his_or.copy()
    
    prefiex='his_'
    
    his['label']=confer_label(his['label'].values)
    
    mt=list(his.mt.unique())
    print(mt)
    

    sum_columns=['model_adcode']
    
    
    def get_last_key_sum(data_,i,key):
        data=data_.copy()
        temp=his[his['mt']==mt[i]]
        for col in sum_columns:
            f=temp[[col,key]].groupby(col)[key].agg('sum')
            data[prefiex+'{}_all_last{}_'.format(key,-i)+col]=data[col].map(f)
        return data
    
    '过去月的销量'   
    for i in range(1,len(mt)+1):
        data= get_last_key_sum(data,-i,'label') 
        
    '过去月的涨幅'    
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,1)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
    
    '过去月的涨幅是正还是负'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去月的涨幅有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all'.format(key)+col]=all_tag_1
        
    '过去月的差分'
    key='label'
    for col in sum_columns:
        for i in range(1,len(mt)):
            data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_'.format(key,i+1)+col].values
            
    '过去月差分的tag'
    for col in sum_columns:
        for i in range(1,len(mt)):
              data[prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
                
    '过去一阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff'.format(key)+col]=all_tag_1 
                
    '过去月的二阶差分'
    for col in sum_columns:
        for i in range(1,len(mt)-1):
            data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col]=data[
                prefiex+'{}_all_last{}_increased_diff'.format(key,i)+col].values-data[
                            prefiex+'{}_all_last{}_increased_diff'.format(key,i+1)+col].values  
    '过去月二阶差分的tag'    
    for col in sum_columns:
        for i in range(1,len(mt)-1):
              data[prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col]=data[prefiex+'{}_all_last{}_increased_diff2'.format(key,i)+col].apply(
          lambda x: 1 if x>0 else 0)
    
    '过去二阶差分有多少为正的'
    for col in sum_columns:
        columns=[]
        for i in range(1,len(mt)-1):
            columns.append(prefiex+'{}_all_last{}_increased_diff_tag2'.format(key,i)+col)
        all_tag_1=np.sum(data[columns].values,axis=1)
        data[prefiex+'{}_all_increased_tag_1all_diff2'.format(key)+col]=all_tag_1 
                    
    


    '销售最大最小'
    key='label'
    for col in sum_columns:
        pre=prefiex+'sales_'+col
        temp=his[[col,key]]
        tmpdf = temp.groupby(col, as_index=False, sort=False)[key].agg(
                        {pre+ '_mean': 'mean',
                         pre + '_max': 'max', pre + '_min': 'min'})
        data=data.merge(tmpdf,on=col,how='left')  
    
    return data

In [25]:
def get_all_features(test_his_set,test_set,train_his_set,train_set,val_his_set,val_set,get_his_features,num_feat_list,cate_feat):
     
    num_feat=num_feat_list.copy()
    
    test1_features1=get_his_features(test_his_set[0],test_set[0])
    for col in list(test1_features1.columns):
        if re.match('his', col)!=None:
            num_feat.append(col)
    
    #test
    test_features_set=[]
    for i in tqdm(range(0,len(test_set))):
        test_features_set.append(get_his_features(test_his_set[i],test_set[i])[num_feat])


    #train
    train_features_set=[]
    for i in tqdm(range(0,len(train_set))):
        train_features_set.append(get_his_features(train_his_set[i],train_set[i])[num_feat])

    #val
    val_features_set=[]
    for i in tqdm(range(0,len(val_set))):
        val_features_set.append(get_his_features(val_his_set[i],val_set[i])[num_feat])
    
    'cat features'
    #test
    test_features_set2=[]
    for i in tqdm(range(0,len(test_set))):
        test_features2=test_set[i][cate_feat]
        test_features_set2.append(test_features2)

    #train
    train_features_set2=[]
    for i in tqdm(range(0,len(train_set))):
        train_features2=train_set[i][cate_feat]
        train_features_set2.append(train_features2)    
    
    #val
    val_features_set2=[]
    for i in tqdm(range(0,len(val_set))):
        val_features2=val_set[i][cate_feat]
        val_features_set2.append(val_features2)    

    'concat features'
    #test
    
    test_features1=pd.concat([i for i in test_features_set],axis=0,sort=True)
    test_features2=pd.concat([i for i in test_features_set2],axis=0,sort=True)
    test_features1.index=np.arange(len(test_features1))
    test_features2.index=np.arange(len(test_features2))
    test1_x=pd.concat([test_features1,test_features2],axis=1)
    print(test1_x.shape)
    #train
    train_features1=pd.concat([i for i in train_features_set],axis=0,sort=True)
    train_features2=pd.concat([i for i in train_features_set2],axis=0,sort=True)
    train_features1.index=np.arange(len(train_features1))
    train_features2.index=np.arange(len(train_features2))
    train_x=pd.concat([train_features1,train_features2],axis=1)[list(test1_x.columns)]
    print(train_x.shape)
    #val
    val_features1 = pd.concat([i for i in val_features_set], axis=0, sort=True)
    val_features2 = pd.concat([i for i in val_features_set2], axis=0, sort=True)
    val_features1.index = np.arange(len(val_features1))
    val_features2.index = np.arange(len(val_features2))
    val_x = pd.concat([val_features1, val_features2], axis=1)[list(test1_x.columns)]
    print(val_x.shape)
    
    
    for i in cate_feat:
        train_x[i]=train_x[i].astype('category')
        test1_x[i]=test1_x[i].astype('category')
        val_x[i]=val_x[i].astype('category')
    
    return train_x,val_x,test1_x,num_feat


num_feat=[]
cate_feat=cate_feat
# cate_feat =cate_feat+ ['last_month_{}'.format(i) for i in range(1,7)]
train_x,val_x,test_x,num_feat_this=get_all_features(test_his_set,test_set,train_his_set,
                                              train_set,val_his_set,val_set,get_his_features_1,num_feat,cate_feat)

  0%|                                                   | 0/12 [00:00<?, ?it/s]

[20, 22, 23, 21, 24, 25, 26, 27]
[20, 22, 23, 21, 24, 25, 26, 27]
[1, 2, 3, 4, 5, 6, 7, 8]


 17%|███████▏                                   | 2/12 [00:00<00:00, 11.64it/s]

[2, 3, 4, 5, 6, 7, 8, 9]
[3, 4, 5, 6, 7, 8, 9, 10]
[4, 5, 6, 7, 8, 9, 10, 11]


 50%|█████████████████████▌                     | 6/12 [00:00<00:00, 12.20it/s]

...

[8, 9, 10, 11, 12, 13, 14, 15]
[9, 10, 11, 12, 13, 14, 15, 16]
[10, 11, 12, 13, 14, 15, 16, 17]


  0%|                                                    | 0/1 [00:00<?, ?it/s]

[11, 12, 13, 14, 15, 16, 17, 18]
[12, 13, 14, 15, 16, 17, 18, 19]
[16, 17, 18, 19, 20, 22, 23, 21]


100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


(484, 71)
(5808, 71)
(484, 71)


## 4月模型

In [26]:
lgb_model=get_model_type(train_x,train_y,val_x,val_y,num_feat_this,cate_feat)
#线下分数
val_pred_4['pred_label']=confer_back_label(lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration_))
train_help.loc[(train_help['mt']==24),'label']=val_pred_4['pred_label'].values  #方便验证
print('12月验证：',score(val_pred_4))
#预测一月
test.loc[(test['mt']==28),'label'] =  confer_back_label(lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration_))
train.loc[(train['mt']==28),'label']=test.loc[(test['mt']==28),'label'].values
print('4月均值',test.loc[(test['mt']==28),'label'].mean())

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.193324	valid_1's rmse: 0.401503
Early stopping, best iteration is:
[55]	training's rmse: 0.24824	valid_1's rmse: 0.382878
                                               f_name   f_score
9           his_label_all_last2_increasedmodel_adcode  0.224848
56                                              model  0.146667
62                                       next_month_4  0.070303
16      his_label_all_last2_increased_tagmodel_adcode  0.054545
23     his_label_all_last1_increased_diffmodel_adcode  0.039394
..                                                ...       ...
14          his_label_all_last7_increasedmodel_adcode  0.000000
7                    his_label_all_last8_model_adcode  0.000000
5                    his_label_all_last6_model_adcode  0.000000
4                    his_label_all_last5_model_adcode  0.000000
35  his_label_all_last6_increased_diff_tagmodel_ad...  0.000000

[71 rows x 2 columns]

# 线下和保存线上 (20个车型)

In [27]:
print('1月均值',test.loc[(test['mt']==25),'label'].mean())
print('4月均值',test.loc[(test['mt']==26),'label'].mean())
print('4月均值',test.loc[(test['mt']==27),'label'].mean())
print('4月均值',test.loc[(test['mt']==28),'label'].mean())

1月均值 250.87011725060526
4月均值 161.1804842865657
4月均值 232.30300950599465
4月均值 206.57001477884594


In [28]:
test.loc[(test['mt']==27),'label'] =  test.loc[(test['mt']==27),'label'].values*0.6
test.loc[(test['mt']==28),'label'] =  test.loc[(test['mt']==28),'label'].values*0.8

In [29]:
print('1月均值',test.loc[(test['mt']==25),'label'].mean())
print('2月均值',test.loc[(test['mt']==26),'label'].mean())
print('3月均值',test.loc[(test['mt']==27),'label'].mean())
print('4月均值',test.loc[(test['mt']==28),'label'].mean())

1月均值 250.87011725060526
2月均值 161.1804842865657
3月均值 139.38180570359688
4月均值 165.2560118230765


In [30]:
#线下分数
val=pd.concat([val_pred_1,val_pred_2,val_pred_3,val_pred_4],axis=0,sort=True)
print('val score',score(val))

val score 0.6060447430593856


In [31]:
sub=test[['id','label']]
sub.columns=['id','forecastVolum']
sub['forecastVolum']=sub['forecastVolum'].apply(lambda x:int(x))
sub.shape
sub[['id','forecastVolum']].to_csv('./submit/sub_zy.csv',index=False)   #我的结果

(1936, 2)